In [2]:
#if needed, you can install all libs here.
# pip install nltk
# pip install vaderSentiment
# pip install pandas
# pip install SQLAlchemy
# pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 19.6 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
#All libs used to analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sqlalchemy import create_engine, text
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [4]:
#Credentials
postgres_dbname = 'clever'
postgres_user = 'clever'
postgres_password = 'clever'
postgres_host = 'postgres_clever'
postgres_port = '5432'

#All avaiable tables into Postgress SQL for checking
avaiable_tables = [
    'fmcsa_complaints',
    'fmcsa_safer_data',
    'fmcsa_company_snapshot',
    'fmcsa_companies',
    'customer_reviews_google',
    'company_profiles_google_maps'
]

In [5]:
#SQL engine, connection and collecting dataframes for analysis
engine = create_engine(
    f'postgresql+psycopg2://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_dbname}'
)

with engine.connect() as conn:
    complaints_df = pd.read_sql('SELECT * FROM fmcsa_complaints', conn)
    safer_data_df = pd.read_sql('SELECT * FROM fmcsa_safer_data', conn)
    companies_df = pd.read_sql('SELECT * FROM fmcsa_companies', conn)
    company_snap = pd.read_sql('SELECT * FROM fmcsa_company_snapshot', conn)
    company_df = pd.read_sql('SELECT * FROM company_profiles_google_maps', conn)
    reviews_df = pd.read_sql('SELECT * FROM customer_reviews_google', conn)
    df_join = pd.read_sql("""SELECT cn.*, s.entity_type, s.legal_name, s.operating_status FROM fmcsa_companies cn
                                     LEFT JOIN fmcsa_safer_data s on cn.usdot_num = s.usdot_num 
                                    """, conn)

In [24]:
def rank_complaints(df, column, city=None, state=None):
    
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df.dropna(subset=[column])

    # Filter by city or state if provided
    if city:
        df = df[df['city'] == city]
    if state:
        df = df[df['state'] == state]
        
    # Sort by column
    df = df.sort_values(by=[column], ascending=False)

    return df

group_df = df_join.groupby(['entity_type','company_name']).max().reset_index()
ranked_total_complaints = rank_complaints(group_df,'total_complaints_2021')
ranked_total_complaints

,entity_type,company_name,usdot_num,user_created,date_created,user_updated,date_updated,company_url,city,state,total_complaints_2021,total_complaints_2022,total_complaints_2023,location,company_type,legal_name,operating_status
80,BROKER,US STANDARD MOVING & STORAGE CORP,3307068,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.655Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.942Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30366362&id2=32319562&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,70,2,0,"MIAMI, FL",0,US STANDARD MOVING & STORAGE CORP,NOT AUTHORIZED
50,BROKER,MOVING APT INC,2247863,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.523Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.616Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30701988&id2=33362625&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,30,12,4,"MIAMI, FL",0,MOVING APT INC,"AUTHORIZED FOR BROKER Property, HHG"
52,BROKER,MOVING SERVICES,2886867,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.533Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.627Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30513053&id2=32723621&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,14,4,3,"MIAMI, FL",0,ADAMS VAN LINES CORP,AUTHORIZED FOR BROKER HHG
92,CARRIER,A1A MOVERS LLC,2821065,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.137Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.226Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30475484&id2=32789951&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,12,12,0,"MIAMI, FL",2,A1A MOVERS LLC,AUTHORIZED FOR HHG
171,CARRIER,EASY ROAD MOVING & STORAGE INC,3491781,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.311Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.400Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30332003&id2=32119219&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,12,11,7,"MIAMI, FL",8,EASY ROAD MOVING & STORAGE INC,AUTHORIZED FOR HHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,CARRIER,C&I EXPRESS TRUCKING,1618312,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.890Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.162Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30818736&id2=29880318&f=search%3D5%26ads%3Da%26state%3DTX%26Submit%3DSearch,DALLAS,TX,0,0,0,"DALLAS, TX",2,WILLIE NICHOLS,NOT AUTHORIZED
134,CARRIER,BRITESTAR TRANSPORT LLC,4006025,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.216Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.299Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=29991137&id2=31620863&f=search%3D5%26ads%3Da%26state%3DFL%26Submit%3DSearch,MIAMI,FL,0,0,0,"MIAMI, FL",1,BRITESTAR TRANSPORT LLC,NOT AUTHORIZED
133,CARRIER,BP AUCTIONS,3715805,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.882Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.154Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30192433&id2=31976619&f=search%3D5%26ads%3Da%26state%3DTX%26Submit%3DSearch,DALLAS,TX,0,0,0,"DALLAS, TX",2,BP AUCTIONS LLC,NOT AUTHORIZED
132,CARRIER,BOLT MOVERS,3545789,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.879Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.151Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp?ads=a&id=30230842&id2=32064715&f=search%3D5%26ads%3Da%26state%3DTX%26Submit%3DSearch,DALLAS,TX,0,0,0,"DALLAS, TX",2,BOLT MOVERS DALLAS LLC,AUTHORIZED FOR HHG


In [17]:
#Ranked reviews by rating descending
def rank_reviews(df, city=None, state=None):
    
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df = df.dropna(subset=['rating'])
    
    # Ensure rating is between 1 and 5
    df = df[(df['rating'].astype(int) >= 1) & (df['rating'].astype(int) <= 5)]
    
    # Filter by city or state if provided
    if city:
        df = df[df['city'] == city]
    if state:
        df = df[df['state'] == state]
        
    # Sort by rating to see the best rating companies
    df = df.sort_values(by=['rating'], ascending=False)

    return df[['name', 'reviews_id', 'rating', 'city', 'state']]


ranked_reviews = rank_reviews(company_df)
ranked_reviews['city'].unique()
# Here you can see the citys and choose one for filtering 

array(['Miami', 'Dallas', 'Seattle'], dtype=object)

In [18]:
ranked_reviews['state'].unique()
# Here you can see the states and choose one for filtering 

array(['Florida', 'Texas', 'Georgia', 'Indiana', 'Oklahoma', 'FL', 'TX',
       'Washington', 'Oregon', 'Pennsylvania', 'WA', 'North Carolina'],
      dtype=object)

In [19]:
rank_reviews(company_df)

,name,reviews_id,rating,city,state
0,Nicolas Boucher P.A,None,5.0,Miami,Florida
1957,Monument Realty - Dallas,None,5.0,Dallas,Texas
1984,"Matt Twomey, Highlands Real Estate",None,5.0,Dallas,Texas
1981,Cecilia Labossiere-Dallas Ft. Worth-Realtor,None,5.0,Dallas,Texas
1980,The Realm Agency,None,5.0,Dallas,Texas
...,...,...,...,...,...
725,Alexandria Real Estate Equities,8.82389343379695E18,1.0,Seattle,Washington
3745,Sun Realty Investments Inc,1.36096E19,1.0,Miami,Florida
1163,Dalrock Properties LLC,None,1.0,Dallas,Texas
1159,First Industrial Realty Trust,1.2609184836918764E19,1.0,Dallas,Texas


In [20]:
rank_reviews(company_df, city = 'Miami')

,name,reviews_id,rating,city,state
0,Nicolas Boucher P.A,None,5.0,Miami,Florida
3444,Mireille P. Segovia Realtor,8.57802E18,5.0,Miami,Florida
3410,Karina Castillo Realtor - PA,None,5.0,Miami,FL
3411,Miami es tu Inversión,None,5.0,Miami,Florida
3413,Lidia Ivonne Garza Realtor,None,5.0,Miami,Florida
...,...,...,...,...,...
3958,"Epic Realty Services, Inc.",6.27208E18,1.0,Miami,Florida
2884,"Core Investment Management, LLC",None,1.0,Miami,Florida
2361,Keller Williams Realty: Ronald Platt,1.68567E19,1.0,Miami,Florida
2274,Cynthia Caridad P.A.,None,1.0,Miami,Florida


In [14]:
import re
import nltk
from nltk.corpus import stopwords

# Ensure you download stopwords the first time
# nltk.download('stopwords')

# Load stop words
stop_words = set(stopwords.words('english'))

# Getting only matching ids
reviews_df = reviews_df.loc[reviews_df['google_id'].str.startswith('0')]
company_df = company_df.loc[company_df['google_id'].str.startswith('0')]

# Merge company and review data
df2 = pd.merge(company_df, reviews_df[['google_id', 'review_text']], on='google_id', how='left')
df2 = df2.loc[~df2['review_text'].isnull()]

# Function to preprocess text (cleaning, stop words removal, etc.)
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    return text

# Preprocess and clean the reviews before analysis
df2['review_text'] = df2['review_text'].apply(preprocess_text)

# Function to analyze sentiment with VADER
def analyze_sentiment(reviews):
    analyzer = SentimentIntensityAnalyzer()

    # Apply sentiment analysis to each review in the 'review_text' column
    reviews['sentiment'] = reviews['review_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

    # Categorize sentiment based on the compound score
    reviews['sentiment_category'] = reviews['sentiment'].apply(
        lambda score: 'positive' if score > 0.05 else ('negative' if score < -0.05 else 'neutral')
    )
    
    return reviews


# Analyze sentiment
df = analyze_sentiment(df2)

# Extract and display relevant information
reviews_analysis = df[['google_id', 'name', 'rating', 'review_text', 'sentiment_category']]
pd.set_option('display.max_colwidth', None)
reviews_analysis.sort_values('rating')


,google_id,name,rating,review_text,sentiment_category
1501,0x864c23be034c2355:0xa6a65be391b3478c,RAIN Realty,3.0,could would give negative thousand license taken every state whole reason hes texas state left business license look dont believe also doesnt pay employees banks million septembers payroll going processed mr rain,negative
1500,0x864c23be034c2355:0xa6a65be391b3478c,RAIN Realty,3.0,rain realty provides brilliant solution selling home low fixed cost method improved profit margin house sold quickly highly recommend service,positive
265,0x549013ff84f9de59:0x4e1ff815e97cc509,Redfin,3.3,cant say enough good things dorothee team knowledge sharp top everything house gained tons attention listed sold quickly well list extremely impressed would recommend anyone plus seller pay cant get better,positive
264,0x549013ff84f9de59:0x4e1ff815e97cc509,Redfin,3.3,rude office phone,negative
263,0x549013ff84f9de59:0x4e1ff815e97cc509,Redfin,3.3,jerks,negative
...,...,...,...,...,...
2768,0x864e99ccd4aad82b:0x5173e239c3d59515,REAL Dallas Properties and Management,5.0,kiana greatest helped find get new place hours able find exactly needed id recommend group anyone looking move dallas area,positive
2769,0x864e99ccd4aad82b:0x5173e239c3d59515,REAL Dallas Properties and Management,5.0,recently pleasure working julissa rodriguez worked overtime help us find home top everything quick responding continues make sure taking care home highly recommend continue working future,positive
2770,0x864e99ccd4aad82b:0x5173e239c3d59515,REAL Dallas Properties and Management,5.0,dealing luna one owners worst experience ive ever moving house never available phone call never respond text messages even emergencies contractors locking us home tell care clients puts act first weve lived month weve multiple incidence water electricity gas set moved dealing water issue supposed handled morning,negative
2773,0x864e99ccd4aad82b:0x5173e239c3d59515,REAL Dallas Properties and Management,5.0,julia amazing helped find dream apartment helpful knowledgeable worked patient picky overly critical went beyond meet expectations owe huge thanks helping find ideal place,positive
